In [ ]:
import pandas as pd
# from tensorflow import keras
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
# from tensorflow.keras.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import tensorflow as tf
import numpy as np
from pandas.tseries.offsets import DateOffset
mpl.rcParams['figure.figsize'] = (15,15)
mpl.rcParams['axes.grid'] = False

In [ ]:
df = pd.read_csv('Aqi_new.csv')
new = pd.read_csv('mumbai-us consulate-air-quality.csv')
new.columns

In [ ]:
new['From Date'] = pd.to_datetime(new['From Date'],infer_datetime_format=True)
new['AQI_calculated'] = new['AQI_calculated'].astype(float)
new.set_index('From Date', inplace=True)
new.head()

In [ ]:
df.info()

In [ ]:
df['From Date'] = pd.to_datetime(df['From Date'],infer_datetime_format=True)

In [ ]:
df = df.replace('None', np.nan)
df = df.fillna(0)

In [ ]:
df1 = df.copy()
df1.drop(['From Date','To Date','Station'], axis = 'columns', inplace=True)
df1.head()
df1 = df1.astype(float)
df1.head()

In [ ]:
df.drop(['To Date'], axis=1)

In [ ]:
def get_PM25_subindex(x):
    if x <= 30:
        return x * 50 / 30
    elif x <= 60:
        return 50 + (x - 30) * 50 / 30
    elif x <= 90:
        return 100 + (x - 60) * 100 / 30
    elif x <= 120:
        return 200 + (x - 90) * 100 / 30
    elif x <= 250:
        return 300 + (x - 120) * 100 / 130
    elif x > 250:
        return 400 + (x - 250) * 100 / 130
    else:
        return 0

df1["PM2.5_SubIndex"] = df1['PM2.5'].apply(lambda x: get_PM25_subindex(x))

## PM10 Sub-Index calculation
def get_PM10_subindex(x):
    if x <= 50:
        return x
    elif x <= 100:
        return x
    elif x <= 250:
        return 100 + (x - 100) * 100 / 150
    elif x <= 350:
        return 200 + (x - 250)
    elif x <= 430:
        return 300 + (x - 350) * 100 / 80
    elif x > 430:
        return 400 + (x - 430) * 100 / 80
    else:
        return 0

df1["PM10_SubIndex"] = df1["PM10"].apply(lambda x: get_PM10_subindex(x))

## SO2 Sub-Index calculation
def get_SO2_subindex(x):
    if x <= 40:
        return x * 50 / 40
    elif x <= 80:
        return 50 + (x - 40) * 50 / 40
    elif x <= 380:
        return 100 + (x - 80) * 100 / 300
    elif x <= 800:
        return 200 + (x - 380) * 100 / 420
    elif x <= 1600:
        return 300 + (x - 800) * 100 / 800
    elif x > 1600:
        return 400 + (x - 1600) * 100 / 800
    else:
        return 0

df1["SO2_SubIndex"] = df1["SO2"].apply(lambda x: get_SO2_subindex(x))

## NOx Sub-Index calculation
def get_NOx_subindex(x):
    if x <= 40:
        return x * 50 / 40
    elif x <= 80:
        return 50 + (x - 40) * 50 / 40
    elif x <= 180:
        return 100 + (x - 80) * 100 / 100
    elif x <= 280:
        return 200 + (x - 180) * 100 / 100
    elif x <= 400:
        return 300 + (x - 280) * 100 / 120
    elif x > 400:
        return 400 + (x - 400) * 100 / 120
    else:
        return 0

df1["NOx_SubIndex"] = df1["NOx"].apply(lambda x: get_NOx_subindex(x))

## CO Sub-Index calculation
def get_CO_subindex(x):
    if x <= 1:
        return x * 50 / 1
    elif x <= 2:
        return 50 + (x - 1) * 50 / 1
    elif x <= 10:
        return 100 + (x - 2) * 100 / 8
    elif x <= 17:
        return 200 + (x - 10) * 100 / 7
    elif x <= 34:
        return 300 + (x - 17) * 100 / 17
    elif x > 34:
        return 400 + (x - 34) * 100 / 17
    else:
        return 0

df1["CO_SubIndex"] = df1["CO"].apply(lambda x: get_CO_subindex(x))

## O3 Sub-Index calculation
def get_O3_subindex(x):
    if x <= 50:
        return x * 50 / 50
    elif x <= 100:
        return 50 + (x - 50) * 50 / 50
    elif x <= 168:
        return 100 + (x - 100) * 100 / 68
    elif x <= 208:
        return 200 + (x - 168) * 100 / 40
    elif x <= 748:
        return 300 + (x - 208) * 100 / 539
    elif x > 748:
        return 400 + (x - 400) * 100 / 539
    else:
        return 0

df1["Ozone_SubIndex"] = df1["Ozone"].apply(lambda x: get_O3_subindex(x))

## NH3 Sub-Index calculation
def get_NH3_subindex(x):
    if x <= 30:
        return x * 50 / 30
    elif x <= 60:
        return 50 + (x - 30) * 50 / 30
    elif x <= 90:
        return 100 + (x - 60) * 100 / 30
    elif x <= 120:
        return 200 + (x - 90) * 100 / 30
    elif x <= 250:
        return 300 + (x - 120) * 100 / 130
    elif x > 250:
        return 400 + (x - 250) * 100 / 130
    else:
        return 0

df1["NH3_SubIndex"] = df1['NH3'].apply(lambda x: get_PM25_subindex(x))

## AQI bucketing
def get_AQI_bucket(x):
    if x <= 50:
        return "Good"
    elif x <= 100:
        return "Satisfactory"
    elif x <= 200:
        return "Moderate"
    elif x <= 300:
        return "Poor"
    elif x <= 400:
        return "Very Poor"
    elif x > 400:
        return "Severe"
    else:
        return np.NaN

df1["Checks"] = (df1["PM2.5_SubIndex"] > 0).astype(int) + \
                (df1["PM10_SubIndex"] > 0).astype(int) + \
                (df1["SO2_SubIndex"] > 0).astype(int) + \
                (df1["NOx_SubIndex"] > 0).astype(int) + \
                (df1["CO_SubIndex"] > 0).astype(int) + \
                (df1["NH3_SubIndex"] > 0).astype(int) + \
                (df1["Ozone_SubIndex"] > 0).astype(int)

df1["AQI_calculated"] = round(df1[["PM2.5_SubIndex", "PM10_SubIndex", "SO2_SubIndex", "NOx_SubIndex",
                                  "CO_SubIndex", "Ozone_SubIndex","NH3_SubIndex"]].max(axis = 1))

df1.loc[df1["PM2.5_SubIndex"] + df1["PM10_SubIndex"] <= 0, "AQI_calculated"] = np.NaN
df1.loc[df1.Checks < 3, "AQI_calculated"] = np.NaN


df1["AQI_bucket_calculated"] = df1["AQI_calculated"].apply(lambda x: get_AQI_bucket(x))
df1[~df1.AQI_calculated.isna()].head(10)
df1.drop(['PM2.5_SubIndex','PM10_SubIndex','SO2_SubIndex','NOx_SubIndex','CO_SubIndex','NH3_SubIndex','Ozone_SubIndex','Checks','AQI_bucket_calculated'],axis=1)

In [ ]:
df = pd.concat([df,df1['AQI_calculated']],axis=1)
df.drop(['To Date'],axis=1,inplace=True)
df.head()

In [ ]:
df2 = df[['From Date', 'Station']].copy()
df.drop(['From Date','Station'],axis=1,inplace=True)
df = df.astype(float)
df.info()

In [ ]:
df = pd.concat([df2,df],axis=1)
df

In [ ]:
# mean = ['PM2.5','PM10',       
#    'NO',           
#    'NO2',         
#     'NOx',          
#     'NH3',        
#     'SO2',        
#     'CO',       
#    'Ozone',   
#    'Benzene',   
#    'Toluene',    
#    'Eth-Benzene', 
# #         'MP-Xylene',
#    'RH',
# #     'SR',
#    'WS',
#    'WD',
#    'BP',
#    'RF',
#    'Xylene',
#    'AT',
#    'TOT-RF','AQI_calculated']
mean = ['AQI_calculated']
mean_df = df.groupby('From Date')[mean].mean()
# mean_df = mean_df[['AQI_calculated']]
mean_df

In [ ]:
newww = pd.concat([new, mean_df])
dfs = newww.sort_values('From Date')

In [ ]:
dfs

In [ ]:
# df_input = mean_df[['PM2.5','PM10',       
#    'NO',           
#    'NO2',         
#     'NOx',          
#     'NH3',        
#     'SO2',        
#     'CO',       
#    'Ozone',   
#    'Benzene',   
#    'Toluene',    
#    'Eth-Benzene', 
# #         'MP-Xylene',
#    'RH',
# #     'SR',
#    'WS',
#    'WD',
#    'BP',
#    'RF',
#    'Xylene',
#    'AT',
#    'TOT-RF','AQI_calculated']]
df_input = dfs[['AQI_calculated']]
df_input.describe()

In [ ]:
df_input.query('AQI_calculated > 400')

In [ ]:
# df_input = df_input[df_input['AQI_calculated'] <= 400]

In [ ]:
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df_input)

In [ ]:
features=data_scaled
target=data_scaled[:,0]

In [ ]:
TimeseriesGenerator(features, target, length=2, sampling_rate=1, batch_size=1)[0]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=123, shuffle = False)

In [ ]:
win_length = 31
batch_size = 3
num_features = 1
# train_generator = TimeseriesGenerator(x_train, y_train, length=win_length, sampling_rate=1, batch_size=batch_size)
test_generator = TimeseriesGenerator(x_test, y_test, length=win_length, sampling_rate=1, batch_size=batch_size)

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('RNN31_1.h5') 

In [ ]:
# model = tf.keras.Sequential()
# model.add(tf.keras.layers.SimpleRNN(128,input_shape = (win_length,num_features),return_sequences = True))
# model.add(tf.keras.layers.LeakyReLU(alpha = 0.5))
# model.add(tf.keras.layers.SimpleRNN(128,return_sequences = True))
# model.add(tf.keras.layers.LeakyReLU(alpha = 0.1))
# model.add(tf.keras.layers.Dropout(0.3))
# model.add(tf.keras.layers.SimpleRNN(64,return_sequences = False))
# model.add(tf.keras.layers.Dropout(0.3))
# model.add(tf.keras.layers.Dense(x_train.shape[1]))

In [ ]:
model.summary()

In [ ]:
model.compile(loss=tf.losses.MeanSquaredError(),
              optimizer=tf.optimizers.Adam(),
              metrics=[tf.metrics.MeanAbsoluteError()])

In [ ]:
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
#                                                     patience=2,
#                                                     mode='min')

# model.compile(loss=tf.losses.MeanSquaredError(),
#               optimizer=tf.optimizers.Adam(),
#               metrics=[tf.metrics.MeanAbsoluteError()])

# history = model.fit_generator(train_generator, epochs=50,
#                     validation_data=test_generator,
#                     shuffle=False,
#                     callbacks=[early_stopping])

In [ ]:
predictions=model.predict(test_generator)

In [ ]:
predictions.shape[0]

In [ ]:
df_pred=pd.concat([pd.DataFrame(predictions), pd.DataFrame(x_test[:,1:][win_length:])],axis=1)
df_pred
# df_pred = pd.DataFrame(predictions)

In [ ]:
rev_trans=scaler.inverse_transform(df_pred)

In [ ]:
df_final=df_input[predictions.shape[0]*-1:]

In [ ]:
df_final.count()

In [ ]:
df_final['Aqi_Pred']=rev_trans[:,0]

In [ ]:
df_final['Aqi_Pred']=rev_trans[:,0]

In [ ]:
df_final[['AQI_calculated','Aqi_Pred']].plot()